In [ ]:
import glob,os
from pathlib import Path
from google.colab import drive
drive.mount('/content/gdrive')

print(os.getcwd())

os.chdir("/content/gdrive/My Drive/Colab Notebooks/srl/txt")


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive
/content


In [ ]:

import nltk.data
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
sent_detector = nltk.data.load('tokenizers/punkt/english.pickle')
import types
import pandas as pd

!pip install allennlp==1.0.0 allennlp-models==1.0.0
from allennlp.predictors.predictor import Predictor



[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
     |████████████████████████████████| 481kB 8.4MB/s 
     |████████████████████████████████| 286kB 19.2MB/s 
     |████████████████████████████████| 675kB 21.9MB/s 
     |████████████████████████████████| 266kB 16.2MB/s 
     |████████████████████████████████| 204kB 30.6MB/s 
     |████████████████████████████████| 61kB 9.3MB/s 
     |████████████████████████████████| 3.8MB 44.2MB/s 
     |████████████████████████████████| 1.1MB 57.0MB/s 
     |████████████████████████████████| 890kB 57.0MB/s 
  Created wheel for overrides: filename=overrides-3.0.0-cp36-none-any.whl size=5669 sha256=141bb749f47ea306fc368a3fec9d0da5c39ced69abcbd34cd5eff5e8959a056e
  Stored in directory: /root/.cache/pip/wheels/6f/1b/ec/6c71a1eb823df7f850d9

In [ ]:
#txt='../sec_cleaning/txt/' #path of txt files from SEC-cleaned for MY LOCAL  ANACONDA


In [ ]:

#os.chdir("../sec_cleaning/txt")
#print(os.getcwd())
#breakpoint()
rid = 0
sentences = {}
filenames=[]

prefixed_files = [filename for filename in glob.glob("*.txt") if filename.startswith("AAL_2")]

#read file path
for text_path in prefixed_files:

    #read all txt file 
    text = Path(text_path).read_text()
    filenames.append(os.path.basename(text_path))
    #breakpoint()
    #print(text)
    #breakpoint()
    #create a dictionary and assign each sentence to a dictionary.
    #dictionary will hold the sentences of each file
    sentences[rid] = []
    text = text.strip()
    #extract each sentence and assign to corresponding the dictionary
    
    text = text.replace("•"," . ")  
    text = text.replace(".",". ")  

    sentences[rid].extend(sent_detector.tokenize(text))
    rid+=1

#print(len(sentences))
#print(sentences[0])
#print(sentences[1])

In [ ]:
import re
causal_verbs = [
    "cause", "causes","caused",
    "result in", "results in", "resulted in", 
    "induce", "induces", "induced", 
    "give rise", "gives rise", "gave rise", "given rise", 
    "generate", "generates", "generated", 
    "bring about", "brings about", "brought about", 
    "lead to", "leads to", "led to", 
    "trigger", "triggers", "triggered", 
    "is linked to", "are linked to", "were linked to", "was linked to", 
    "brings forth", 
    "leads up to", "lead up to", "led up to",
    "triggers off", "trigger off", "triggered off", 
    "bring on", "brings on", "brought on",
    "affect", "affects", "affected",
    "effect", "effects", "effected",
    "instigate", "instigates", "instigated",
    "stimulate", "stimulated", "stimulates",
    "engender", "engendered","engenders", 
    "impact", "impacts", "impacted",
    "become ground", "became ground", "become grounds", "became grounds"
]
reverse_causal_verbs = [
    "stems from", "stemmed from", "stem from",
    "brought on by", 
    "caused by", 
    "affected by", "effected by",
    "resulted from", "results from", "as a result of",
    "emerged from", "emerges from", "carried out by", "invoked by", "occurred because", "in consequence"
]


all_verbs = causal_verbs+reverse_causal_verbs

#print(all_verbs)
causal_re = re.compile(" | ".join(causal_verbs), re.IGNORECASE)
reverse_causal_re = re.compile(" | ".join(reverse_causal_verbs), re.IGNORECASE)

In [ ]:
use_chunks = False
chunking_method = "nltk"
min_chunk_strlen = 6

# The following is a temporary simple definition of get_ce
def get_ce(part1, part2, use_chunks=False, chunking_method="nltk", min_chunk_strlen=6):
    pairs = []
    pairs.append((part1,part2))
    return pairs
    
def get_ce_pairs(sent, rec_id, use_chunks=False, chunking_method="nltk", min_chunk_strlen=6):
    cees = []
    sent = re.sub('[\\s+]', ' ', sent)
    #if len(sent) > 200:
    #    return None
    parts = causal_re.split(sent)
    for i in range(0, len(parts) - 1):
        for (cause, effect) in get_ce(parts[i],parts[i+1],use_chunks, chunking_method, min_chunk_strlen):
            cees.append((cause, effect,rec_id, sent))
    parts = reverse_causal_re.split(sent)
    for i in range(0, len(parts) - 1):
        for (cause, effect) in get_ce(parts[i+1],parts[i],use_chunks, chunking_method, min_chunk_strlen):
            cees.append((cause, effect,rec_id, sent))
    return cees

In [ ]:

"""

old parse function and grammar it is using

"""


grammar = r"""
          NP: {<NN|NNS>+<><NN|NNS>+}              # Verb and its suroundings
          """

grammar = r"""
  NP: {<PDT|DT|JJ|PP|NN.*>+}          # Chunk sequences of DT, JJ, NN
  PP: {<IN|DT|PDT><NP>}               # Chunk prepositions followed by NP
  VP: {<VB.*><NP|PP|CLAUSE>+(<\.>)?$} # Chunk verbs and their arguments
  CLAUSE: {<NP><VP>}           # Chunk NP, VP
  """
cp = nltk.RegexpParser(grammar, loop=1)

def get_verb_left_right(sent):
    verb = None
    left_np = None
    right_np = None
    sent_pos = nltk.pos_tag(nltk.word_tokenize(sent))
    parsed_sent = cp.parse(sent_pos)
    #print(parsed_sent)
    for node in parsed_sent:
        if node.__class__.__name__=='tuple':
            continue
        if node.label()=='NP':
            left_np = node
        if node.label()=='VP':  
            vp = node
            verb = vp[0][0]
            if len(vp)>1:
                if vp[1].__class__.__name__!='tuple' and vp[1].label()=='PP':
                    verb += " " + vp[1][0][0]
            for vnode in vp[1]:
                if vnode.__class__.__name__=='tuple':
                    continue
                if vnode.label()=='NP':
                    right_np = vnode
    if verb and right_np and left_np:
        return (verb,' '.join(v[0] for v in left_np.leaves()),' '.join(v[0] for v in right_np.leaves()))
    else:
        return None



import re
#tokenLeft = 'ARG0'
#tokenRight = 'ARG1'
#tokenVerb = 'V'

#patterns for tokens
patternVerb = r'\[V: (.*?)\]'
patternLeft = r'\[ARG0: (.*?)\]'
patternRight= r'\[ARG1: (.*?)\]'

#pattern = rf"\[{tokenLeft}: (.*?)\].*\[{tokenVerb}: (.*?)\].*\[{tokenRight}: (.*?)\]"


# link for the srl features
predictor = Predictor.from_path("https://storage.googleapis.com/allennlp-public-models/bert-base-srl-2020.03.24.tar.gz")

def get_verb_left_right2(sentence):
    verb = None
    left_np = None
    right_np = None
    print("input sentence: {0}".format(sentence))

    # if sentence doesn't contain the causal verbs, return
    if not any((word in sentence) for word in all_verbs)  :
      missed_verbs_writer.writerow(["NONE",sentence])
      return

    #parse the sentence
    result = predictor.predict(
      sentence=sentence
    )

    #list for parse trees that contain the causal verbs and arguments
    verb_arguments_list =[]

    for elem in result['verbs']:

      #extract verb from the parse tree
      m = re.search(patternVerb, elem['description'], re.IGNORECASE)

      if(m):
        verb = m.group(1)
        
        #if verb is a causal verb, continue
        if(verb in all_verbs):
          
          #extract left argument from the sentence
          m = re.search(patternLeft, elem['description'], re.IGNORECASE)
          if(m):
            left_np = m.group(1)

            #extract right argument from the sentence
            m = re.search(patternRight, elem['description'], re.IGNORECASE)
            if(m):
              right_np = m.group(1)
              
              # when verb, left argument, right argument are all extracted, add them to the list
              verb_arguments_list.append((verb,left_np,right_np))
              print("######pushed {0}".format((verb,left_np,right_np)))
        else:
          print("!! '{0}' not found in the causal verbs list ".format(verb))
          #write the discarded verb to the file
          missed_verbs_writer.writerow([verb,sentence])

    
    return verb_arguments_list




100%|██████████| 406056588/406056588 [00:06<00:00, 59127073.08B/s]


In [ ]:
import csv
Sentence_output_hilal_2 = "Sentence_2.csv"
missed_verbs = "missed_verbs.csv"

csvOutput = open(Sentence_output_hilal_2,"w") #<===open and write to a csv file  and New rows will be appended.
missed_verbs_csv = open(missed_verbs,"w") #<===open and write to a csv file  and New rows will be appended.


csvWriter = csv.writer(csvOutput, quoting = csv.QUOTE_NONNUMERIC)
missed_verbs_writer = csv.writer(missed_verbs_csv, quoting = csv.QUOTE_NONNUMERIC)

csvWriter.writerow(["file name","direction","verb","cause","effect","sent"])
missed_verbs_writer.writerow(["verb","sentence"])


count = 0
#breakpoint()
for rid in sentences:
    for sent in sentences[rid]:
        #parts = get_verb_left_right(sent)
        part_list = get_verb_left_right2(sent)
        
        if(part_list):
          for parts in part_list:
            if parts:
                verb = parts[0]
                if verb.lower() in causal_verbs:
                    count += 1
                    cause = parts[1]
                    effect = parts[2]
                    print("Found causal verb and cause-effect pairs: ([{}],[{}],[{}]) from sentence: {}".format(verb,cause,effect,sent))
                    csvWriter.writerow([filenames[rid],"forward",verb,cause,effect,sent])
                if verb.lower() in reverse_causal_verbs:
                    count += 1
                    cause = parts[2]
                    effect = parts[1]
                    csvWriter.writerow([filenames[rid],"reverse",verb,cause,effect,sent])

                    print("Found reverse causal verb and cause-effect pair: ([{}],[{}],[{}]) from sentence: {}".format(verb,cause,effect,sent))        
                    print("Found {} cause-effect pairs.".format(count))
                

print("end of loop")

csvOutput.close()
missed_verbs_csv.close()

input sentence: .
input sentence: below are certain risk factors that may affect our business, results of operations and financial condition, or the trading price of our common stock or other securities.
!! 'may' not found in the causal verbs list 
######pushed ('affect', 'certain risk factors', 'our business , results of operations and financial condition , or the trading price of our common stock or other securities')
Found causal verb and cause-effect pairs: ([affect],[certain risk factors],[our business , results of operations and financial condition , or the trading price of our common stock or other securities]) from sentence: below are certain risk factors that may affect our business, results of operations and financial condition, or the trading price of our common stock or other securities.
input sentence: we caution the reader that these risk factors may not be exhaustive.
input sentence: we operate in a continually changing business environment, and new risks and uncertainti